# research_code

> A selection of useful code chunks for analysing THz-TDS data.

In [27]:
#| hide
from fastcore.test import *
import fastcore.docments as docment
from nbdev.showdoc import DocmentTbl

In [2]:
#| default_exp research_code

In [3]:
#| export 
import numpy as np

In [4]:
#| export
def exp_decay(x: np.ndarray, # Input array that contains the indpendent variable.
    Amplitude: np.single, # Pre-factor of the exponential decay.
    tau: np.single, # Decay constant.
    x0: np.single, # Offset for the independent variable.
    ) -> np.ndarray: # A new float array containing the exponentially decaying function.
    '''
    Single exponentially decaying function.
    '''
    decayOutput = Amplitude * np.exp(-(x-x0)/tau)
    
    return decayOutput

In [5]:
#| export
def biexp_decay(x:np.ndarray, # Input array that contains the indpendent variable.
    Amplitude1: np.single, # Prefactor of the first exponential decay.
    tau1: np.single, # Decay constant of the first exponential term.
    Amplitude2: np.single, # Prefactor of the second exponential decay.
    tau2: np.single, # Decay constant of the second exponential term.
    x0: np.single, # Offset for the indpendent variable.
    ) -> np.ndarray: # A new float array containing the exponentially decaying function.
    '''
    Bi-exponentially decaying function.
    '''
    decayOutput = Amplitude1 * np.exp(-(x-x0)/tau1) + Amplitude2 * np.exp(-(x-x0)/tau2)
    
    return decayOutput

In [6]:
#| export
def gaussian(x:np.ndarray, # Input array that contains the indpendent variable.
    Amplitude: np.single, # Prefactor of the gaussian.
    sigma: np.single, # Standard deviation of the gaussian.
    x0: np.single, # Offset for the indpendent variable.
    ) -> np.ndarray: # A new float array containing the gaussian function.
    '''
    Gaussian function.
    '''
    outputGaussian = Amplitude * np.exp(-((x-x0)/sigma)**2)

    return outputGaussian

In [7]:
#| export
def convolved_exp_decay(x:np.ndarray, # Input array that contains the indpendent variable.
    Amplitude: np.single, # Prefactor of the exponential decay.
    tau: np.single, # Decay constant.
    sigma: np.single, # Standard deviation of the gaussian
    x0: np.single, # Offset for the indpendent variable.
    y0: np.single, # Offset in y.
    ) -> np.ndarray: # A new float array containing the exponentially decaying function.
    '''
    Single exponentially decaying function convolved with a gaussian.
    '''
    L = len(x) # length of the independent variable.

    exp_term = exp_decay(x,Amplitude,tau,x0) # The exponential decay.

    gaussian_term = gaussian(x,1.0,sigma,x0) # The gaussian part, setting the amplitude normalised to 1.0.
    
    convolved = np.convolve(exp_term,gaussian_term,mode='full')[0:L] # convolved function.

    convolved *= (Amplitude-y0)/np.max(convolved) # Normalise the function so the maximum is at Amplitude
    
    return convolved + y0

In [29]:
#| export
def convolved_biexp_decay(x:np.ndarray, # Input array that contains the indpendent variable.
    Amplitude1:np.single, # Prefactor of the first exponential decay.
    tau1:np.single, # Decay constant of the first exponential term.
    Amplitude2:np.single, # Prefactor of the second exponential decay.
    tau2:np.single, # Decay constant of the second exponential term.
    sigma:np.single, # Standard deviation of the gaussian
    x0:np.single, # Offset for the indpendent variable.
    y0:np.single, # Offset in y.
    ) ->np.ndarray: # A new float array containing the exponentially decaying function.
    '''
    Bi-exponentially decaying function convolved with a gaussian.
    '''    
    L = len(x) # length of the independent variable.

    biexp_term = biexp_decay(x,Amplitude1,tau1,Amplitude2,tau2,x0) # The biexponential decay.

    gaussian_term = gaussian(x,1.0,sigma,x0) # The gaussian part, setting the amplitude normalised to 1.0.
    
    convolved = np.convolve(biexp_term,gaussian_term,mode='full')[0:L] # convolved function.

    convolved *= (Amplitude1+Amplitude2-y0)/np.max(convolved) # Normalise the function so the maximum is at Amplitude
    
    return convolved + y0

## Functions for use with un-excited THz-TDS data

In [9]:
#| export
def data_reformatter(df):

    """
    The purpose of this function is to repackage or change the format of the data, 
    such that rather than having two columns with a large number of rows, we have
    different columns for each scan. The data is also augmented with two additional 
    rows - the mean and the std of the scans.

    """
    data = df.to_numpy()

    Ntot,_ = np.shape(data) # totoal number of rows

    delays = np.unique(data[:,0]) # unique delay positions in mm
    
    delays = 2*delays/(consts.c) * 1e-3 * 1e12
    
    N = len(delays) # number of delays per scan

    M = int(Ntot/N) # number of scans

    df2 = pd.DataFrame(np.reshape(data[:,1],(M,N)).T) # reshape the data and put it into a dataframe

    col_names = {i:f'scan{i}' for i in range(M)} # create an array with names like 'scan0, scan1, ....'
    
    df2 = df2.rename(columns=col_names) # rename the columns

    mean_col = df2.mean(axis=1) # calculate a new column containing the mean value for each delay

    std_col = df2.std(axis=1) # calculate a new column containing the std value for each delay

    df2['mean'] = mean_col # augment the dataframe with a new column containing the mean

    df2['std'] = std_col # augment the dataframe with a new column containing the std
    
    df2.insert(loc=0,column='delay',value=delays) # add a new column at the start of the DataFrame containing the delays

    return df2

In [34]:
DocmentTbl(convolved_biexp_decay)

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| x | ndarray | Input array that contains the indpendent variable. |
| Amplitude1 | float32 | Prefactor of the first exponential decay. |
| tau1 | float32 | Decay constant of the first exponential term. |
| Amplitude2 | float32 | Prefactor of the second exponential decay. |
| tau2 | float32 | Decay constant of the second exponential term. |
| sigma | float32 | Standard deviation of the gaussian |
| x0 | float32 | Offset for the indpendent variable. |
| y0 | float32 | Offset in y. |
| **Returns** | **ndarray** | **A new float array containing the exponentially decaying function.** |

In [10]:
#| hide
from nbdev.showdoc import *

In [11]:
#| export
def foo(): pass

In [12]:
#| hide
import nbdev; nbdev.nbdev_export()